# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --mem-fraction-static 0.8"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-04-26 01:50:18] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=31263, mem_fraction_static=0.8, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=428289369, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None,

[2025-04-26 01:50:28 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-26 01:50:28 TP0] Init torch distributed begin.
[2025-04-26 01:50:28 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-26 01:50:28 TP0] Load weight begin. avail mem=60.58 GB


[2025-04-26 01:50:28 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-26 01:50:29 TP0] Using model weights format ['*.safetensors']


[2025-04-26 01:50:29 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.63it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.63it/s]



...

[2025-04-26 01:50:30 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-04-26 01:50:30] INFO:     Started server process [909081]
[2025-04-26 01:50:30] INFO:     Waiting for application startup.
[2025-04-26 01:50:30] INFO:     Application startup complete.
[2025-04-26 01:50:30] INFO:     Uvicorn running on http://0.0.0.0:31263 (Press CTRL+C to quit)


[2025-04-26 01:50:31] INFO:     127.0.0.1:46268 - "GET /v1/models HTTP/1.1" 200 OK
[2025-04-26 01:50:31] INFO:     127.0.0.1:46274 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-26 01:50:31 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-26 01:50:34] INFO:     127.0.0.1:46288 - "POST /generate HTTP/1.1" 200 OK
[2025-04-26 01:50:34] The server is fired up and ready to roll!


Server started on http://localhost:31263


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-26 01:50:36 TP0] Prefill batch. #new-seq: 1, #new-token: 37, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-26 01:50:37 TP0] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 6.00, #queue-req: 0
[2025-04-26 01:50:37] INFO:     127.0.0.1:40060 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-04-26 01:50:37 TP0] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 5, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-26 01:50:37 TP0] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 119.53, #queue-req: 0


[2025-04-26 01:50:37 TP0] Decode batch. #running-req: 1, #token: 128, token usage: 0.01, gen throughput (token/s): 131.08, #queue-req: 0


[2025-04-26 01:50:37 TP0] Decode batch. #running-req: 1, #token: 168, token usage: 0.01, gen throughput (token/s): 139.11, #queue-req: 0
[2025-04-26 01:50:38] INFO:     127.0.0.1:40060 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-04-26 01:50:38] INFO:     127.0.0.1:40060 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-04-26 01:50:38 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0
Yes, it is a test. I'm here

 to assist you with any questions or tasks you might have. Feel free to ask[2025-04-26 01:50:38 TP0] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, gen throughput (token/s): 127.96, #queue-req: 0


 anything you like, and I'll do my best to

 help you.

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-04-26 01:50:38 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-26 01:50:38 TP0] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 113.36, #queue-req: 0


[2025-04-26 01:50:38] INFO:     127.0.0.1:40060 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-04-26 01:50:38 TP0] Decode batch. #running-req: 0, #token: 0, token usage: 0.00, gen throughput (token/s): 139.61, #queue-req: 0
[2025-04-26 01:50:38 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-26 01:50:39] INFO:     127.0.0.1:40060 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-04-26 01:50:39] INFO:     127.0.0.1:40068 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-26 01:50:39] INFO:     127.0.0.1:40068 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-26 01:50:39 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 48, token usage: 0.00, #running-req: 0, #queue-req: 0


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-04-26 01:50:39 TP0] Decode batch. #running-req: 2, #token: 48, token usage: 0.00, gen throughput (token/s): 61.50, #queue-req: 0


[2025-04-26 01:50:39 TP0] Decode batch. #running-req: 1, #token: 75, token usage: 0.00, gen throughput (token/s): 201.06, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-04-26 01:50:42] INFO:     127.0.0.1:40068 - "GET /v1/batches/batch_2a7931e5-9eea-48ad-86b7-747dae962ac8 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-04-26 01:50:42] INFO:     127.0.0.1:40068 - "GET /v1/files/backend_result_file-fb8f8f51-4a68-49d2-8892-0f1658d0c389/content HTTP/1.1" 200 OK


[2025-04-26 01:50:42] INFO:     127.0.0.1:40068 - "DELETE /v1/files/backend_result_file-fb8f8f51-4a68-49d2-8892-0f1658d0c389 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-04-26 01:50:42] INFO:     127.0.0.1:32892 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-26 01:50:42] INFO:     127.0.0.1:32892 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-26 01:50:42 TP0] Prefill batch. #new-seq: 10, #new-token: 300, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-04-26 01:50:42 TP0] Prefill batch. #new-seq: 10, #new-token: 310, #cached-token: 30, token usage: 0.01, #running-req: 10, #queue-req: 0


[2025-04-26 01:50:42 TP0] Decode batch. #running-req: 20, #token: 1243, token usage: 0.06, gen throughput (token/s): 220.14, #queue-req: 0


[2025-04-26 01:50:52] INFO:     127.0.0.1:51558 - "GET /v1/batches/batch_a8bbe411-13cf-499d-ab17-2143001566f1 HTTP/1.1" 200 OK


[2025-04-26 01:50:55] INFO:     127.0.0.1:51558 - "GET /v1/batches/batch_a8bbe411-13cf-499d-ab17-2143001566f1 HTTP/1.1" 200 OK


[2025-04-26 01:50:58] INFO:     127.0.0.1:51558 - "GET /v1/batches/batch_a8bbe411-13cf-499d-ab17-2143001566f1 HTTP/1.1" 200 OK


[2025-04-26 01:51:01] INFO:     127.0.0.1:51558 - "GET /v1/batches/batch_a8bbe411-13cf-499d-ab17-2143001566f1 HTTP/1.1" 200 OK


[2025-04-26 01:51:04] INFO:     127.0.0.1:51558 - "GET /v1/batches/batch_a8bbe411-13cf-499d-ab17-2143001566f1 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-04-26 01:51:07] INFO:     127.0.0.1:49810 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-26 01:51:07] INFO:     127.0.0.1:49810 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-26 01:51:08 TP0] Prefill batch. #new-seq: 43, #new-token: 710, #cached-token: 742, token usage: 0.03, #running-req: 0, #queue-req: 0
[2025-04-26 01:51:08 TP0] Prefill batch. #new-seq: 97, #new-token: 2910, #cached-token: 428, token usage: 0.06, #running-req: 43, #queue-req: 477


[2025-04-26 01:51:09 TP0] Decode batch. #running-req: 140, #token: 5323, token usage: 0.26, gen throughput (token/s): 61.10, #queue-req: 4860
[2025-04-26 01:51:09 TP0] Prefill batch. #new-seq: 22, #new-token: 660, #cached-token: 110, token usage: 0.29, #running-req: 139, #queue-req: 4838


[2025-04-26 01:51:09 TP0] Prefill batch. #new-seq: 2, #new-token: 60, #cached-token: 10, token usage: 0.48, #running-req: 160, #queue-req: 4836
[2025-04-26 01:51:09 TP0] Decode batch. #running-req: 161, #token: 12209, token usage: 0.60, gen throughput (token/s): 13163.36, #queue-req: 4836


[2025-04-26 01:51:10 TP0] Decode batch. #running-req: 161, #token: 18649, token usage: 0.91, gen throughput (token/s): 17508.73, #queue-req: 4836
[2025-04-26 01:51:10 TP0] Decode out of memory happened. #retracted_reqs: 24, #new_token_ratio: 0.6007 -> 0.9297


...

[2025-04-26 01:51:17 TP0] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 5, token usage: 0.58, #running-req: 141, #queue-req: 4056
[2025-04-26 01:51:17] INFO:     127.0.0.1:37684 - "POST /v1/batches/batch_4a51d6a0-e1eb-4aef-9514-add1f955425c/cancel HTTP/1.1" 200 OK


[2025-04-26 01:51:17 TP0] Prefill batch. #new-seq: 58, #new-token: 7465, #cached-token: 214, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-04-26 01:51:18 TP0] Decode batch. #running-req: 57, #token: 7628, token usage: 0.37, gen throughput (token/s): 8104.94, #queue-req: 0


[2025-04-26 01:51:18 TP0] Decode batch. #running-req: 5, #token: 748, token usage: 0.04, gen throughput (token/s): 4044.34, #queue-req: 0


[2025-04-26 01:51:20] INFO:     127.0.0.1:37684 - "GET /v1/batches/batch_4a51d6a0-e1eb-4aef-9514-add1f955425c HTTP/1.1" 200 OK


[2025-04-26 01:51:20] INFO:     127.0.0.1:37684 - "DELETE /v1/files/backend_input_file-b0ff5c45-6619-421c-989c-ed92f8789085 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)